In [1]:
from pathlib import Path
from validphys.loader import _get_nnpdf_profile
from validphys.api import API
import numpy as np
import pandas as pd
from validphys.convolution import central_predictions

profile = _get_nnpdf_profile()
yaml_db = Path(profile["data_path"]) / "yamldb"

The `yaml_db` folder is a temporary thing as it contains files that look like:

```yaml
conversion_factor: 1.0
operands:
- - NMC_NC_EM_D_F2
- - NMC_NC_EM_P_F2
operation: RATIO
target_dataset: NMCPD
```

This information will eventually be part of the new commondata format of course.

The `operation` is applied to the first level of the list while the second level is just concatenated. This is necessary since `pineappl` fktables might contain one layer of concatenation which is already done for the "classic" fktables.

The `pineappl` fktables will live inside the appropiate `theory_xxx` folder `/pineappls`.

In [2]:
# Test them all
if True:
    from yaml import safe_load
    pdf = API.pdf(pdf="NNPDF40_nnlo_as_01180")
    all_res = []
    nnpdf40_runcard = safe_load(Path("/home/juacrumar/NNPDF-testing/nnpdf/n3fit/NNPDF40_with_pineappl.yml").read_text())
    #nnpdf40_runcard = safe_load(Path("/mount/storage/Academic_Workspace/NNPDF/source/nnpdf/n3fit/NNPDF40_with_pineappl.yml").read_text())
    for d in nnpdf40_runcard["dataset_inputs"]:
        target_ds = d["dataset"]
        cfac = d.get("cfac", [])
        old_ds = API.dataset(dataset_input={"dataset": target_ds, "cfac": cfac + ["oldmode"]}, theoryid=200, use_cuts="internal")
        ds = API.dataset(dataset_input={"dataset": target_ds, "cfac": cfac}, theoryid=200, use_cuts="internal")
        new_cp = central_predictions(ds, pdf)
        cp = central_predictions(old_ds, pdf)
        all_res.append(pd.concat([new_cp, cp, new_cp/cp], axis=1, keys=["vp", "pine", f"ratio {target_ds}, {cfac}"]))
        
    for i in all_res:
        mean_ratio = i[i.columns[2]].mean()
        if not (0.9 < mean_ratio < 1.1):
            print(i)

LHAPDF 6.4.0 loading /usr/share/lhapdf/LHAPDF/NNPDF40_nnlo_as_01180/NNPDF40_nnlo_as_01180_0000.dat
NNPDF40_nnlo_as_01180 PDF set, member #0, version 1
                vp        pine ratio CMS_2JET_7TEV, ['QCD']
                 0           0                            0
data                                                       
0     1.765957e+02  174.848775                     1.009991
1     2.039113e+01   25.082712                     0.812955
2     3.014533e+00    4.401240                     0.684928
3     5.180528e-01    0.906572                     0.571441
4     1.003950e-01    0.204804                     0.490201
5     2.106041e-02    0.049915                     0.421927
6     4.759396e-03    0.013042                     0.364939
7     1.061027e-03    0.003342                     0.317436
8     2.308831e-04    0.000836                     0.276226
9     4.694783e-05    0.000194                     0.241540
10    8.208930e-06    0.000039                     0.211423
11    1.8

In [6]:
target_ds = "CMS_TTBAR_2D_DIFF_MTT_TRAP_NORM"
cfac = ["QCD"] # ["NRM"]
old_ds = API.dataset(dataset_input={"dataset": target_ds, "cfac": cfac + ["oldmode"]}, theoryid=200, use_cuts="internal")
ds = API.dataset(dataset_input={"dataset": target_ds, "cfac": cfac}, theoryid=200, use_cuts="internal")

In [7]:
# Let's try to get a prediction out of it
pdf = API.pdf(pdf="NNPDF40_nnlo_as_01180")
new_cp = central_predictions(ds, pdf)
cp = central_predictions(old_ds, pdf)
pd.concat([new_cp, cp, cp/new_cp, new_cp/cp], axis=1, keys=["pine", "vp", "ratio vp/ratio", "ratio pine/vp"])

,pine,vp,ratio vp/ratio,ratio pine/vp
,0,0,0,0
data,,,,
0,0.003383,0.003382,0.999938,1.000062
1,0.003004,0.002982,0.992669,1.007385
2,0.002128,0.002114,0.993233,1.006813
3,0.000740,0.000741,1.001264,0.998738
4,0.002754,0.002664,0.967190,1.033923
5,0.002355,0.002363,1.003314,0.996697
6,0.001637,0.001637,0.999900,1.000100
7,0.000580,0.000578,0.995907,1.004110


In [5]:
pine_fkspec = ds.fkspecs[0]
old_fkspec = old_ds.fkspecs[0]

In [6]:
import pineappl
pines = [pineappl.fk_table.FkTable.read(i.as_posix()) for i in pine_fkspec.fkpath]
# Inspect the pineappl prediction
res_pine = []
pp = pines[0]
lpdf = pdf.load()

for p in pines:
    res_pine.append(p.convolute_with_one(2212, lpdf.central_member.xfxQ2))
total_pine = np.concatenate(res_pine)
print(total_pine)

[1.74819495e+00 1.98846641e-01 2.90280375e-02 4.90519957e-03
 9.44769537e-04 1.98407976e-04 4.36709986e-05 9.70543391e-06
 2.08104072e-06 4.10372403e-07 6.98121341e-08 1.55353480e-08
 1.66466499e-09 1.40177702e+00 1.94396456e-01 3.20542598e-02
 5.94480545e-03 1.21760469e-03 2.58607189e-04 5.58446250e-05
 1.15951595e-05 2.22080859e-06 3.73179113e-07 5.08574384e-08
 5.02048228e-09 3.70769697e-01 6.25529562e-02 1.19041186e-02
 2.43795694e-03 5.15963446e-04 1.09640302e-04 2.23113284e-05
 4.17790750e-06 4.48599784e-07 3.19089676e-08 1.86894005e-09
 1.84336831e-01 3.50614302e-02 7.16582800e-03 1.52306228e-03
 3.23843890e-04 6.68743273e-05 1.24317740e-05 1.99734202e-06
 2.53770589e-07 3.96763628e-09 1.43188858e-02 3.06196032e-03
 6.49614036e-04 1.30079488e-04 2.35675017e-05 3.61760539e-06
 4.21371227e-07 7.53287048e-09]
LHAPDF 6.4.0 loading all 101 PDFs in set NNPDF40_nnlo_as_01180
NNPDF40_nnlo_as_01180, version 1; 101 PDF members


In [7]:
# Let's inspect the content of the old fktables, remove the cfactor for now
from validphys.fkparser import load_fktable
old_fkspec.cfactors = False
old_fktabledata = load_fktable(old_fkspec)

In [8]:
print(f"hadronic?: {old_fktabledata.hadronic}")
print(f"Q: {old_fktabledata.Q0}")
print(f"n: {old_fktabledata.ndata}")
print(f"xgrid shape: {old_fktabledata.xgrid.shape}")
#old_fktabledata.sigma

hadronic?: True
Q: 1.65
n: 54
xgrid shape: (50,)


In [9]:
# First read the metadata that vp `FKTableData` needs and that all subgrids share
Q0 = np.sqrt(pp.muf2())
xgrid = pp.x_grid()
# Hadronic means in practice that not all luminosity combinations are just electron X proton
hadronic = not all(-11 in i for i in pp.lumi())
# Now prepare the concatenation of grids
fktables = []
for p in pines:
    tmp = p.table().T/p.bin_normalizations()
    fktables.append(tmp.T)
fktable = np.concatenate(fktables, axis=0)
ndata = fktable.shape[0]

In [10]:
pp.lumi()

[(100, 100),
 (100, 21),
 (100, 200),
 (100, 203),
 (100, 208),
 (100, 215),
 (100, 224),
 (100, 235),
 (100, 103),
 (100, 108),
 (100, 115),
 (100, 124),
 (100, 135),
 (21, 21),
 (21, 200),
 (21, 203),
 (21, 208),
 (21, 215),
 (21, 224),
 (21, 235),
 (21, 103),
 (21, 108),
 (21, 115),
 (21, 124),
 (21, 135),
 (200, 200),
 (200, 203),
 (200, 208),
 (200, 215),
 (200, 224),
 (200, 235),
 (200, 103),
 (200, 108),
 (200, 115),
 (200, 124),
 (200, 135),
 (203, 203),
 (203, 208),
 (203, 215),
 (203, 224),
 (203, 235),
 (203, 103),
 (203, 108),
 (203, 115),
 (203, 124),
 (203, 135),
 (208, 208),
 (208, 215),
 (208, 224),
 (208, 235),
 (208, 103),
 (208, 108),
 (208, 115),
 (208, 124),
 (208, 135),
 (215, 215),
 (215, 224),
 (215, 235),
 (215, 103),
 (215, 108),
 (215, 115),
 (215, 124),
 (215, 135),
 (224, 224),
 (224, 235),
 (224, 103),
 (224, 108),
 (224, 115),
 (224, 124),
 (224, 135),
 (235, 235),
 (235, 103),
 (235, 108),
 (235, 115),
 (235, 124),
 (235, 135),
 (103, 103),
 (103, 108),


In [11]:
df.columns

NameError: name 'df' is not defined

In [ ]:
# Now let's try to join the fktable, luminosity and xgrid into a pandas dataframe
# keeping compatibility with validphys and, hopefully, 50% of my own sanity

# Step 1), make the luminosity into a 14x14 mask for the evolution basis
eko_numbering_scheme = (22, 100, 21, 200, 203, 208, 215, 224, 235, 103, 108, 115, 124, 135)
# note that this is the same ordering that was used in fktables
co = []
for i, j in pp.lumi():
    # Ask where this would fall in a 14x14 matrix
    idx = eko_numbering_scheme.index(i)
    jdx = eko_numbering_scheme.index(j)
    co.append(idx*14 + jdx)
    
# Step 2) prepare the indices for the dataframe
xi = np.arange(len(xgrid))
ni = np.arange(ndata)
mi = pd.MultiIndex.from_product([ni, xi, xi], names=["data", "x1", "x2"])

# Step 3) Now play with the array until we flatten it in the right way?
# The fktables for pineappl have this extra factor of x...
# The output of pineappl is (ndata, flavours, x, x)
lf = len(co)
xfktable = fktable.reshape(ndata, lf, -1)/(xgrid[:,None]*xgrid[None,:]).flatten()
fkmod = np.moveaxis(xfktable, 1, -1)
fkframe = fkmod.reshape(-1, lf)

# Uff, big
df = pd.DataFrame(fkframe, index=mi, columns=co)

from validphys.convolution import central_hadron_predictions
from validphys.coredata import FKTableData
fk = FKTableData(sigma=df, ndata=ndata,  Q0=Q0, metadata=None, hadronic=True, xgrid=xgrid)
central_hadron_predictions(fk, pdf)

In [ ]:
# Create a luminosity tensor and check that the results are correct
from validphys.pdfbases import evolution

evol_basis = (
    "photon",
    "singlet",
    "g",
    "V",
    "V3",
    "V8",
    "V15",
    "V24",
    "V35",
    "T3",
    "T8",
    "T15",
    "T24",
    "T35",
)
total_pdf = evolution.grid_values(pdf, evol_basis, xgrid, [Q0]).squeeze()[0]/xgrid
print(total_pdf.shape)
lumi = np.einsum('ij,kl->ikjl', total_pdf, total_pdf)
lumi_masked = lumi[flavour_map]
print(fktable.shape)
print(lumi_masked.shape)
res = np.einsum('ijkl,jkl->i', fktable, lumi_masked)
#pd.concat([pd.DataFrame(res), cp, pd.DataFrame(res)/cp,  ], axis=1)

In [ ]:
xfktable.reshape(48,91,-1).shape

In [ ]:
from validphys.fkparser import open_fkpath, _parse_string, _parse_header, _build_sigma
from validphys.fkparser import _parse_flavour_map, _parse_hadronic_fast_kernel
try:
    f.close()
except:
    pass
f = open_fkpath(old_fkspec.fkpath)
line_and_stream = enumerate(f, start=1)
lineno, header = next(line_and_stream)
res = {}
while True:
    marker, header_name = _parse_header(lineno, header)
    if header_name == "FastKernel":
        break
    if header_name == "FlavourMap":
        out, lineno, header = _parse_flavour_map(line_and_stream)
    else:
        out, lineno, header = _parse_string(line_and_stream)
    res[header_name] = out   

In [ ]:
res["FlavourMap"].shape

In [ ]:
i_hate_pandas = _parse_hadronic_fast_kernel(f)

In [ ]:
i_hate_pandas

In [ ]:
old_fktabledata.sigma